In [0]:
%sql
alter table event add angle number;

In [1]:
%sql
update event set angle = 
  SDO_UTIL.CONVERT_UNIT(
       atan((y_rel_m + (7.32/2)) / x_rel_m) - atan((y_rel_m - (7.32/2)) / x_rel_m)
    , 'Radian', 'Degree'
    );

In [2]:
%sql
alter table event add angle_bucket number;

In [3]:
%sql
update event set angle_bucket = floor(angle);

In [4]:
%sql
create table event_train as select event_id, angle, head, from_corner, set_piece, fast_break, is_goal from event where to_char(match_date, 'YYYY') in ('2017', '2018')
and penalty = 'N' and own_goal = 'N';

In [5]:
%sql
create table model_parameters(setting_name varchar2(30), setting_value varchar2(4000));

In [6]:
%sql
insert into model_parameters(setting_name, setting_value) values ('ALGO_NAME', 'ALGO_GENERALIZED_LINEAR_MODEL');

In [7]:
%sql
call dbms_data_mining.create_model('XG_MODEL', 'CLASSIFICATION', 'EVENT_TRAIN', 'EVENT_ID', 'IS_GOAL', 'MODEL_PARAMETERS');

In [8]:
%sql
alter table event add predicted_goal varchar2(1);

In [9]:
%sql
alter table event add xg number;

In [10]:
%sql
update event
set predicted_goal = PREDICTION(XG_MODEL USING *)
, xg = prediction_probability(XG_MODEL, 'Y' USING *);

In [11]:
%sql
commit;